# Step 1: Install Python packages

In [12]:
import os
import html5lib
import pandas as pd
from selenium import webdriver                   
from selenium.webdriver.common.keys import Keys   
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup as bs    

# Step 2: Preparation

In [37]:
wsj = "https://www.wsj.com/market-data/stocks?mod=nav_top_subsection"
# Locate Driver in system
current_path = os.getcwd()
# save the .exe file under the same directory of the web-scrape python script.
Path = os.path.join(current_path, "chromedriver")
# Initialize Chrome driver and start browser session controlled by automated test software.
driver = webdriver.Chrome(executable_path='chromedriver')
# Get the URL
driver.get(wsj)
timeout = 30
try:
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, "root")))
except TimeoutException:
    driver.quit()
table = driver.find_element_by_id("root").get_attribute('outerHTML')

In [38]:
tables  = pd.read_html(table)
tables

[              Unnamed: 0      Last     Chg  %Chg
 0                   DJIA  28344.29  195.65  0.70
 1       Nasdaq Composite  11384.93   52.44  0.46
 2                S&P 500   3429.83   21.20  0.62
 3  DJ Total Stock Market  35205.95  277.82  0.80
 4           Russell 2000   1615.36   33.39  2.11
 5         NYSE Composite  13053.18  103.53  0.80
 6           Barron's 400    747.31    9.53  1.29
 7        CBOE Volatility     27.82   -0.14 -0.50
 8           DJIA Futures  28175.00  180.00  0.64
 9        S&P 500 Futures   3414.25   21.35  0.63,
   Unnamed: 0_level_0               NYSE             Nasdaq
               Issues Unnamed: 1_level_1 Unnamed: 2_level_1
            Issues At Unnamed: 1_level_2 Unnamed: 2_level_2
         Share Volume Unnamed: 1_level_3 Unnamed: 2_level_3
 0          Advancing               2182               2533
 1          Declining                860                913
 2          Unchanged                 78                 81
 3              Total        

In [47]:
wsj_tables =[df for df in tables]

In [51]:
sp500_sectors_df = wsj_tables[5]
sp500_sectors_df.columns = ["S&P 500 & Sectors", "% Change"]
sp500_sectors_df.reset_index(drop=True, inplace=True)
sp500_sectors_df

,S&P 500 & Sectors,% Change
0,S&P 500,0.58
1,Communication Services,-0.20
2,Consumer Discretionary,-0.06
3,Consumer Staples,0.43
4,Energy,1.11
5,Financials,1.36
6,Health Care,0.09
7,Industrials,0.89
8,Information Technology,0.36
9,Materials,1.25


In [54]:
sp500_sectors_html = sp500_sectors_df.to_html()
sp500_sectors_table = str(sp500_sectors_html)
sp500_sectors_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>S&amp;P 500 &amp; Sectors</th>\n      <th>% Change</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>S&amp;P 500</td>\n      <td>0.58</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Communication Services</td>\n      <td>-0.20</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Consumer Discretionary</td>\n      <td>-0.06</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Consumer Staples</td>\n      <td>0.43</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Energy</td>\n      <td>1.11</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Financials</td>\n      <td>1.36</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Health Care</td>\n      <td>0.09</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>Industrials</td>\n      <td>0.89</td>\n    </tr>\n    <tr>\n      <th>8</th>\n      <td>Information Technology</td>\n 

In [197]:
# The index of the links needed
# 0	S&P 500
# 1	Communication Services
# 2	Consumer Discretionary
# 3	Consumer Staples
# 4	Energy
# 5	Financials
# 6	Health Care
# 7	Industrials
# 8	Information Technology
# 9	Materials
# 10	Real Estate
# 11	Utilities
sp500_links = [(driver.find_elements_by_partial_link_text(sector)) for sector in sp500_sectors_df["S&P 500 & Sectors"]]
sp500_link[0], sp500_link[0], sp500_link[0],
# for sector in sp500_sectors_df["S&P 500 & Sectors"]: 
#     sector = driver.find_elements_by_partial_link_text(sector)
#     for links in sector: 
#          links.get_attribute("href")
# links

# sp500_link = [links.get_attribute("href") for links in sp500_links[0]]
# Communication Services
# Consumer Discretionary
# Consumer Staples
# Energy
# Financials
# Health Care
# Industrials
# Information Technology
# Materials
# Real Estate
# Utilities
# .find_elements_by_tag_name('a')
# .get_attribute("href")
# sp500_links = [sp500_link.find_elements_by_class_name("WSJTables--table__cell--2u6629rx WSJTables--is-first--f1NiiZJm WSJTheme--table__cell--3njwWeaF ") for sp500_link in page_links]
# sp500_links
# sp500_element.get_attribute("href")

'https://www.wsj.com/market-data/quotes/index/US/S&P%20US/SPX?mod=md_usstk_overview_quote'

In [35]:
driver.quit()

In [ ]:
# def wsjTable(url):
#     tables = pd.read_html(url, attrs={'class': "WSJTables--table--1QzSOCfq WSJTheme--table--Tzxxxq4J "})
# #     sp500_sector_df = tables[0]
# #     sp500_sector_df.columns = ["S&P 500", "% Change"]
# #     sp500_sector_html =  sp500_sector_df.to_html()
# #     sp500_sector_list = [str(sp500_sector_html)]
# #     return sp500_sector_list
# wsjTable("http://www.wsj.com/market-data/stocks?mod=nav_top_subsection")

In [ ]:
# def initScrape():
#     wsj = "https://www.wsj.com/market-data/stocks?mod=nav_top_subsection"
#     wsjTable(wsj)
# #     # Locate Driver in system
# #     current_path = os.getcwd()
# #     # save the .exe file under the same directory of the web-scrape python script.
# #     Path = os.path.join(current_path, "chromedriver")
# #     # Initialize Chrome driver and start browser session.
# #     service = Service(Path)
# #     service.start()
# #     # Open a blank Chrome Browser and inform it is controlled by automated test software.
# #     browser = webdriver.Remote(service.service_url)
# #     # It implicity waits 30 seconds to perform all task in the browsers session. 
# #     browser.implicitly_wait(30) 
# #     scrapeWsj(wsj)
# #     broswer.quit()
#     wsj_dict = {"S&P 500 Sectors": sp500_sector_list}
#     return wsj_dict

# Step 3: Find the IDs of the items we want to scrape for

# Step 4: Techniques to make more human-like web-scrapers

In [ ]:
# If the website detects us as a web-scraper, it will cut our connection so we cannot pull more data and have to re-start our scraper. This largely impacts the efficiency of the scraper and involves a lot of manual interference. There are a few techniques we can use to make the scraper more human-like:
# (1) Randomize the sleep time
# This can be easily implemented as below wherever needed:
#sleep for sometime between 5 and 8 seconds
# time.sleep(random.uniform(5,8))
# (2) Randomize the user agent for the web browser
# This is also easy and can be added to the browser options as below:
# ua = UserAgent()
# userAgent = ua.random
# Firefox_options = webdriver.FirefoxOptions()
# Firefox_options.add_argument(f’user-agent={userAgent}’)
# browser = webdriver.Firefox(executable_path = DRIVER_BIN, options=Firefox_options)
# (3) Use dynamic proxy/IP
# This requires more work than the above two. Usually free proxies are not stable and most of them don’t respond to requests, so we need to first a free proxy that responds to our requests. This website (also named as “url” in the script below) provides a lot of free proxies which we scrape down for our use. We will use Python BeautifulSoup package to scrape a list of proxies, and use Python requests package to test whether the proxy responds to our requests to the link.
# def get_proxy(link):
#     url = "https://www.sslproxies.org/"
#     r = requests.get(url)
#     soup = BeautifulSoup(r.content, 'html5lib')
#     proxies_list = list(map(lambda x: x[0]+':'+x[1], list(zip(map(lambda x: x.text, soup.findAll('td')[::8]), map(lambda x: x.text, soup.findAll('td')[1::8])))))
#     while 1:
#         try:
#             selected_ip = choice(proxies_list)
#             proxy = {'https': selected_ip, 'http': selected_ip}
#             headers = {'User-Agent': ua.random}
#             print('Using proxy:{}'.format(proxy))
#             r = requests.request('get', link, proxies=proxy, headers=headers, timeout=5)
#             break
#         except:
#             pass
        
#     return proxy
# We then add the working proxy to the browser option, similar to how we added the fake user agent:
# link = "https://www.expedia.com"
# proxy = get_proxy(link)
# Firefox_options.add_argument('--proxy-server=%s' % proxy)
# browser = webdriver.Firefox(executable_path = DRIVER_BIN, options=Firefox_options)

# Step 5: The full code that runs the scraper and save the data to .csv files

In [ ]:
# def scrapeWsj(wsj): 
#     tables = pd.read_html(wsj)
#     sp500_sector_df = tables[0]
#     sp500_sector_df.columns = ["S&P 500", "% Change"]
#     sp500_sector_html =  sp500_sector_df.to_html()
#     sp500_sector_list = [str(sp500_sector_html)]
# #     browser.get(wsj)
#     return sp500_sector_list

In [ ]:
# initScrape()